In [35]:
import pandas as pd
import numpy as np
import sys
import nbimporter
import sources.endomondolib as endo
import sources.pysparkconvenience as ps
from numpy import array
from math import sqrt
from pyspark.sql.functions import *
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql import DataFrameReader
from pyspark.sql import SQLContext
from IPython.display import display, HTML
from pyspark.sql.functions import col
from pyspark.mllib.linalg import Vectors

from pyspark.ml.feature import VectorAssembler

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.mpl_style = 'default'

%load_ext autotime


The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 11 ms


In [3]:
#####RDD Function Area######
def print_rdd(rdd):
    for x in rdd.collect():
        print x
        
#create pyspark dataframe from rdd
def df_from_rdd(rdd):
    pass

#create pyspark dataframe from csv
def df_from_csv(csv_file):
    text = sc.textFile(csv_file)\
        .map(lambda line: line.split(","))
    #didn't work with take(1). believe returns 
    #different object then first()
    schema = text.first()
    data = text.filter(lambda x: x != schema)
    df = sqlContext.createDataFrame(data, schema)
    return df


time: 5.15 ms


In [4]:
####FUNCTION AREA###########

#get workout info for workoutid
def get_rdd_workout_from_workoutid(workoutid, df_workouts):
    filtered_df = df_workouts.filter(df_workouts['workoutid']==workoutid)
    return filtered_df
    

##given user id, returns dataframe of workouts by that user
##NOT DONE
def get_workouts_for_userid(userid, df_workout):
    #get workoutIds done by this user
    workout_ids = df_workout\
        .filter(df_workout['userid']==userid)\
        .select('workoutid').flatMap(lambda x:x).collect()
    print workout_ids
    df = df_sport.where(col("workoutid").isin(workout_ids))
    print df.show()
    print df.count()
    return userid

#get workoutids for user
def get_rdd_workoutids_for_userid(userid, df_workouts):
    filtered_df = df_workouts.filter(df_workouts['id']==userid)
    workoutids = filtered_df.select('workoutid').flatMap(list)
    return workoutids

#convert type of a column of a df
def convertColumn(df, name, new_type):
    df_1 = df.withColumnRenamed(name, "swap")
    return df_1.withColumn(name, df_1["swap"].cast(new_type)).drop("swap")

# #fills 
# def fill_na_with_values(dict){
    
# }

time: 12.9 ms


In [5]:
url = 'postgresql://34.199.246.37:5432/endomondo'
properties = {'user':'endomondo', 'password':'End0m0ndo'}

df_workouts = DataFrameReader(sqlContext).jdbc(
   url='jdbc:%s' % url, table='public.run_sub_with_hr', properties=properties
)

df_users = DataFrameReader(sqlContext).jdbc(
   url='jdbc:%s' % url, table='public.run_sub_with_hr_users', properties=properties
)



time: 7.01 s


In [6]:
wout_ids = get_rdd_workoutids_for_userid(1295505, df_workouts).collect()
wout_id = wout_ids[0]
df_workouts_for_user = get_rdd_workout_from_workoutid(wout_id, df_workouts)
print df_workouts_for_user.count()

273
time: 2.8 s


# sample clustering section

In [7]:
#get columns of the dataframe we want to cluster
df_workouts_for_user.columns

['time',
 'altitude',
 'heart_rate',
 'latitude',
 'longitude',
 'speed',
 'workoutid',
 'id']

time: 11.3 ms


In [8]:
#fill nans with a value
df4 = df_workouts_for_user.na.fill(0)

time: 12.1 ms


In [33]:
#make each row in the dataframe into feature vector for clustering
#assembler = VectorAssembler(
#     inputCols=["gender", "foo", "bar"], outputCol="features")
#final_df = assembler.transform(encoded_dff)

vecAssembler = VectorAssembler(inputCols=['altitude',\
 'heart_rate',\
 'latitude',\
 'longitude',\
 'speed'], outputCol="features")
final_df = vecAssembler.transform(df4)
final_df = final_df.select('features')
print final_df.show()
#vecAssembler.transform(df4).head(3).features
#new_df = vecAssembler.transform(df4)

+--------------------+
|            features|
+--------------------+
|[0.0,85.0,55.7095...|
|[0.0,85.0,55.7095...|
|[0.0,85.0,55.7095...|
|[0.0,85.0,55.7094...|
|[0.0,89.0,55.7094...|
|[0.0,97.0,55.7093...|
|[0.0,97.0,55.7093...|
|[0.0,100.0,55.709...|
|[0.0,104.0,55.709...|
|[0.0,104.0,55.709...|
|[0.0,104.0,55.709...|
|[0.0,110.0,55.709...|
|[0.0,113.0,55.709...|
|[0.0,115.0,55.709...|
|[0.0,116.0,55.709...|
|[0.0,115.0,55.709...|
|[0.0,117.0,55.709...|
|[0.0,117.0,55.709...|
|[0.0,117.0,55.709...|
|[0.0,122.0,55.709...|
+--------------------+
only showing top 20 rows

None
time: 577 ms


In [37]:
final_df1 = final_df.rdd\
    .map(lambda row : Vectors.dense([item for item in row]))
final_df1.collect()

[DenseVector([0.0, 85.0, 55.7095, 37.7582, 0.0]),
 DenseVector([0.0, 85.0, 55.7095, 37.7582, 0.0]),
 DenseVector([0.0, 85.0, 55.7095, 37.7583, 0.0]),
 DenseVector([0.0, 85.0, 55.7095, 37.7583, 0.0]),
 DenseVector([0.0, 89.0, 55.7094, 37.7586, 0.0]),
 DenseVector([0.0, 97.0, 55.7094, 37.7588, 0.0]),
 DenseVector([0.0, 97.0, 55.7093, 37.7589, 0.0]),
 DenseVector([0.0, 100.0, 55.7093, 37.7592, 0.0]),
 DenseVector([0.0, 104.0, 55.7092, 37.7593, 0.0]),
 DenseVector([0.0, 104.0, 55.7092, 37.7594, 0.0]),
 DenseVector([0.0, 104.0, 55.7092, 37.7595, 0.0]),
 DenseVector([0.0, 110.0, 55.7092, 37.7597, 0.0]),
 DenseVector([0.0, 113.0, 55.7093, 37.7599, 0.0]),
 DenseVector([0.0, 115.0, 55.7095, 37.76, 0.0]),
 DenseVector([0.0, 116.0, 55.7095, 37.7599, 0.0]),
 DenseVector([0.0, 115.0, 55.7096, 37.7597, 0.0]),
 DenseVector([0.0, 117.0, 55.7097, 37.7596, 0.0]),
 DenseVector([0.0, 117.0, 55.7097, 37.7595, 0.0]),
 DenseVector([0.0, 117.0, 55.7097, 37.7594, 0.0]),
 DenseVector([0.0, 122.0, 55.7097, 37.75

time: 616 ms


In [42]:
# Build the model (cluster the data)
clusters = KMeans.train(final_df1, 2, maxIterations=10, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
# def error(point):
#     center = clusters.centers[clusters.predict(point)]
#     return sqrt(sum([x**2 for x in (point - center)]))

# WSSSE = final_df1.map(lambda point: error(point)).reduce(lambda x, y: x + y)
# print("Within Set Sum of Squared Error = " + str(WSSSE))
# Save and load model
#clusters.save(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")
#sameModel = KMeansModel.load(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")

time: 623 ms


In [44]:
clusters.clusterCenters

[array([   0.        ,  108.48      ,   55.70952189,   37.75904871,    0.        ]),
 array([   0.        ,  151.49193548,   55.70951535,   37.75903852,    0.        ])]

time: 8.2 ms


In [ ]:
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)